### This is a notebook to track a bug found when merging two LnasFormat

The bug apparently happens after loading the merged lnas.

Expected behavior:
- The number of triangles of the merged lnas must be the sum of both lnas triangles count
- The number of triangles of a surface from this mesh must be equal after loading

Observed behavior:
- Full mesh triangles count remain the same after loading from file, and equals to the sum of both lnas
- The number of triangles of a surface from this mesh **doubles** after loading

Steps to reproduce:
- Combine two lnas
- Save to file
- Load mesh from file
- Triangle count for surface 'm_000_inside' is not the same after loading


In [49]:
import numpy as np
from lnas import LnasFormat
import pathlib

meshes_path = pathlib.Path("./fixtures/lnas")

mesh = LnasFormat.from_file(meshes_path / "G100.lnas")
negative_mesh = LnasFormat.from_file(meshes_path / "G100_negative.lnas")

mesh.geometry.triangles.shape, negative_mesh.geometry.triangles.shape 

((41045, 3), (1840, 3))

Merge lnas

In [50]:
NORMAL_OFFSET = - 0.125

for surface_name in negative_mesh.surfaces.keys():
    tri_normals = negative_mesh.geometry_from_surface(surface_name).normals.copy()
    tri_verts = negative_mesh.geometry_from_surface(surface_name).triangle_vertices.copy()
    tri_verts += np.repeat(tri_normals, 3, axis=0).reshape(-1, 3, 3) * NORMAL_OFFSET

    new_verts = tri_verts.copy().reshape(-1, 3)
    new_verts = np.unique(new_verts, axis=0)

    new_triangles = []

    for tri in tri_verts:
        tri_index = []
        for v in tri:
            matches = np.all(new_verts == v, axis=1)
            indices = np.where(matches)
            tri_index.append(indices[0][0])
        tri_index.reverse()
        new_triangles.append(np.array(tri_index))

    new_triangles = np.array(new_triangles, dtype=np.uint32) + len(mesh.geometry.vertices)

    new_tri_indices = np.arange(
        mesh.geometry.triangles.shape[0], mesh.geometry.triangles.shape[0] + new_triangles.shape[0]
    )
    mesh.surfaces[surface_name + "_inside"] = new_tri_indices

    mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
    mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

    mesh.geometry._update_triangles_vertices()
    mesh.geometry._update_normals()
    mesh.geometry._update_areas()
    mesh.geometry._update_vertices_normals()

filename = meshes_path / "G100.merged.lnas"
export_path = meshes_path / "G100.merged.stl"

mesh.to_file(filename)
geometry = mesh.geometry_from_surface("m_000_inside")

print("Number of triangles before export: ", mesh.geometry.triangles.shape)
print("Number of triangles reference surface: ", geometry.triangles.shape)

Number of triangles before export:  (42885, 3)
Number of triangles reference surface:  (312, 3)


In [51]:
loaded_mesh = LnasFormat.from_file(meshes_path / "G100.merged.lnas")
loaded_geometry = loaded_mesh.geometry_from_surface("m_000_inside")

print("Number of triangles after loading: ", loaded_mesh.geometry.triangles.shape)
print("Number of triangles reference surface: ", loaded_geometry.triangles.shape)

Number of triangles after loading:  (42885, 3)
Number of triangles reference surface:  (624, 3)
